<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agent_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interactive LLM Agent for Continual Summarization

## Requirements

FLAML requires `Python>=3.7`. To run this notebook example, please install flaml with the [openai] option, and feedparser:
```bash
pip install flaml[autogen] feedparser
```

In [1]:
# %pip install flaml[autogen]==2.0.0rc1 feedparser

## Set your API Endpoint


The [`config_list_gpt4_gpt35`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_gpt4_gpt35) function tries to create a list of gpt-4 and gpt-3.5 configurations using Azure OpenAI endpoints and OpenAI endpoints. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.

In [3]:
from flaml import oai
config_list = oai.config_list_from_models(model_list=["gpt-3.5-turbo-0613"])

## Continaul Research Digest via `LeanrningAgent` and `TeachingAgent`


In [7]:
from flaml.autogen.agent import LearningAgent, TeachingAgent
import openai
from flaml import oai

def LLM_related(input_string):
    if "Large Language Models" in input_string or "LLM" in input_string or "GPT" in input_string:
        return True
    else:
        return False

import feedparser

research_teacher = TeachingAgent(name="research_teacher", human_input_mode="NEVER")
research_teacher.setup_learning(
    learning_constraints={"learning_trigger": True, "cpu": 1},
    learning_objectives="""Condense the provided data, which consists of titles and abstracts of research papers, into a research digest.
    Create a single bullet point for each entry, ensuring clarity and coherence.
    """,
    learning_results=" ",
    # learning_func=oai.summarize,
)
# get data and add to research_teacher
ml_feed = feedparser.parse("http://export.arxiv.org/rss/cs.LG")
ai_feed = feedparser.parse("http://export.arxiv.org/rss/cs.AI")
ml_data, ai_data = [], []

# for demo purpose, only use 3 entries from ml_feed and ai_feed
for entry in ml_feed.entries:
    title_and_abstract = f"Title: {entry.title}. \n Abstract: {entry.summary}"
    if LLM_related(title_and_abstract):
        ml_data.append(title_and_abstract)
research_teacher.add_data(ml_data)
for entry in ai_feed.entries:
    title_and_abstract = f"Title: {entry.title}. \n Abstract: {entry.summary}"
    if LLM_related(title_and_abstract):
        ai_data.append(entry.summary)
KEY_LOC = "notebook/"
research_teacher.add_data(ai_data)
config_list = oai.config_list_from_models(key_file_path=KEY_LOC)
research_learner = LearningAgent(name="research_learner", model="gpt-3.5-turbo-0613", config_list=config_list)
research_learner.receive(research_teacher.generate_init_prompt(), research_teacher)


research_teacher (to research_learner):

Message content:
learning_func: None
learning_objectives: Condense the provided data, which consists of titles and abstracts of research papers, into a research digest.
    Create a single bullet point for each entry, ensuring clarity and coherence.
    
learning_constraints: {'learning_trigger': True, 'cpu': 1}
learning_results:  
data4learning: []

--------------------------------------------------------------------------------
*********Current learning results of the learner*********
  
**************************************************
research_learner (to research_teacher):

Message content:
learning_results:  
is_data_size_feasible: <function LearningAgent._is_data_size_feasible_oai at 0x7f9cd0faa040>

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...
research_teacher (to research_learner):

Message content:
learning_results:  
data4learning: